In [0]:
%run  ../connector/sql_connector

In [0]:
dbutils.widgets.text("full_load", "DELTA")

In [0]:
from pyspark.sql.functions import current_timestamp, sum, count, avg,date_format, round

def tabela_delta_existe(uc_table):
    try:
        spark.sql(f"DESCRIBE {uc_table}")
        return True
    except Exception as e:
        return False

In [0]:
from delta.tables import *

filtro = "data_venda  >= current_date() - 90" if dbutils.widgets.get("full_load") == "DELTA" else "1=1"


merge_key = "target.ano_mes = updates.ano_mes"

dataframe = spark.table("""dev.silver.stage__farmacia""") \
    .withColumn("ano_mes", date_format("data_venda", "yyyy-MM")) \
    .filter(filtro) \
    .groupBy("ano_mes") \
    .agg(count("venda_id").alias("qtd_vendas"),
        sum("valor_total").alias("valor_total_vendas"),
        round((sum("valor_total") / count("venda_id")),2).alias("media_venda")         
    ) \
    .withColumn("load_ts", current_timestamp()) 

print("Ingestion mode: ", dbutils.widgets.get("full_load"))
if tabela_delta_existe("dev.gold.performance_farmacia"):
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)

    DeltaTable.forName(spark, "hive_metastore.default.performance_farmacia").alias("target") \
        .merge(dataframe.alias("updates"), merge_key) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    dataframe \
        .write \
        .format("delta") \
        .saveAsTable("dev.gold.performance_farmacia")